<p align="center">
  <img src="Graphics/Episode IV.png" />
</p>

## (0) The Unified Planning Framework (UPF)

### (0.1) Learning Outcomes

In this tutorial, we will cover:
* How can we integrate PDDL with Python using the Unified Planning Framework?
* How can we build some of the planning problems we've already explored with the UPF?

<p align="center">
  <img src="Graphics/UPF.png" width=700/>
</p>

## (1) Integrating Classical Planning into Python with the UPF

The [Unified Planning Framework](https://www.ai4europe.eu/research/ai-catalog/unified-planning-framework) is a Python library developed by the [AIPlan4EU](https://www.aiplan4eu-project.eu/) project, that makes it easy to formulate planning problems and to invoke automated planners directly from Python scripts. This library will allow us to produce PDDL files in a more convenient way and run them through AI planners with ease.

<p align="center">
  <img src="Graphics/UPF_Diagram.png" />
</p>

In order to install the library (and the Fast Downward planner), please run the following commands from your computer's terminal:
```
pip install unified-planning==0.4.2.362.dev1
pip install up-fast-downward==0.0.4
```

We'll begin by revisiting the simple world domain that we've seen before, and understanding how we can represent it using the UPF.

### (1.1) UPF Basics

Let's remind ourselves of the following problem: imagine we are in a room where there is an apple on a shelf one side of the room, a robot in the middle of the room, and a table on the other side of the room from the shelf. This world looks as follows:
<p align="center">
  <img src="Graphics/simple_world.png" />
</p>
We want to command the robot to perform a simple task like “take the apple from the shelf and put it on the table”. How can we represent this world and this task using the UPF?

First, we should import the necessary libraries into this notebook:

In [ ]:
from unified_planning.shortcuts import *
from unified_planning.io import PDDLWriter

Now that we have UPF loaded, we can begin describing the problem. In this case, we don't need to declare the domain name at the start, nor do we need to declare any requirements. We can go straight to defining the new types that we need for this problem, which we did in TA#2 using PDDL as follows:
```
    (:types apple location robot - object)
```

With the UPF, we can declare the types in our domain as follows:

In [ ]:
Apple = UserType("Apple")
Robot = UserType("Robot")
Location = UserType("Location")

These types are all automatically derived from `object`, but if we wanted to derive them from a different type we could add an argument called `father`, for example:
```
Robot = UserType("Robot", father=Location)
```
This would derive the `Robot` type as a child of the `Location` type. We won't use this functionality just yet, but it'll be useful for our next example.

Next, we can define our predicates for this domain, which we did in PDDL as follows:
```
    (:predicates (On ?a - apple ?l - location)(Holding ?r - robot ?a - apple)(At ?r - robot ?l - location))
```
With the UPF, we can write the following:

In [ ]:
At = Fluent("At", BoolType(), robot=Robot, location=Location)
On = Fluent("On", BoolType(), apple=Apple, location=Location)
Holding = Fluent("Holding", BoolType(), robot=Robot, apple=Apple)

The predicates are declared using the `Fluent` function, as predicates are just boolean fluents (a fluent is a variable in a planning problem - it can be boolean, numeric, etc.), with the `BoolType()` type to indicate that they have boolean values. We will see later how to use this to declare numeric fluents as well.

Next up, we have our actions, which were defined as follows in PDDL:
```
(:action pick
    :parameters (?a - apple ?r - robot ?l - location)
    :precondition (and (On ?a ?l)
                       (not (Holding ?r ?a))
                       (At ?r ?l))
    :effect (and (not (On ?a ?l))
                 (Holding ?r ?a))
    )

    (:action place
    :parameters (?a - apple ?r - robot ?l - location)
    :precondition (and (not (On ?a ?l))
                       (Holding ?r ?a)
                       (At ?r ?l))
    :effect (and (On ?a ?l)
                 (not (Holding ?r ?a)))
    )

    (:action move
    :parameters (?r - robot ?from ?to - location)
    :precondition (and (At ?r ?from)
                       (not (At ?r ?to)))
    :effect (and (At ?r ?to)
                 (not (At ?r ?from)))
    )
```     

Now with the UPF, we can define them like this:

In [ ]:
move = InstantaneousAction("move", r=Robot, l_from=Location, l_to=Location)
l_from = move.parameter("l_from")
l_to = move.parameter("l_to")
r = move.parameter("r")
move.add_precondition(At(r,l_from))
move.add_precondition(Not(At(r,l_to)))
move.add_effect(At(r,l_to),True)
move.add_effect(At(r,l_from),False)

pick = InstantaneousAction("pick", a=Apple, r=Robot, l=Location)
a = pick.parameter("a")
r = pick.parameter("r")
l = pick.parameter("l")
pick.add_precondition(At(r,l))
pick.add_precondition(On(a,l))
pick.add_precondition(Not(Holding(r,a)))
pick.add_effect(On(a,l),False)
pick.add_effect(Holding(r,a),True)

place = InstantaneousAction("place", a=Apple, r=Robot, l=Location)
a = pick.parameter("a")
r = pick.parameter("r")
l = pick.parameter("l")
place.add_precondition(At(r,l))
place.add_precondition(Not(On(a,l)))
place.add_precondition(Holding(r,a))
place.add_effect(On(a,l),True)
place.add_effect(Holding(r,a),False)

As you can see, we use the `InstantaneousAction` function to create a new action, and we can add its parameters using `.parameter`, its preconditions using `.add_precondition`, and its effects using `.add_effect`. Be aware that some planners can't handle negative preconditions (i.e. using `Not` in the preconditions), so just be aware of that when choosing a planner for a problem! When declaring effects, we simply set the boolean value of the predicates of interest to either `True` or `False`, as you can see above (i.e. we don't need to use the `Not` operator there).

At this point we've established our domain, and now we can start building our problem. Looking back to our PDDL problem file, we see that we started by declaring the domain and the objects in our problem:
```
(:domain simple)
	(:objects
        Apple - apple
        Robot - robot
        Shelf Table - location
    )
```
With the UPF we didn't need to declare a domain name, but we will declare a name in order to define a new problem:

In [ ]:
problem_simple = Problem("simple")

The `Problem` class will store all of the information needed to represent and solve our planning problems. We'll start by declaring our objects and adding them to the problem:

In [ ]:
robot0 = Object("robot0", Robot)
apple0 = Object("apple0", Apple)
shelf = Object("shelf", Location)
table = Object("table", Location)

problem_simple.add_objects([robot0,apple0,shelf,table])

Notice that we add our objects to the problem using the `.add_objects` method; without it, our objects won't be attached to our problem, even after we declared them as above. The same will go for pretty much every other thing in PDDL. For example, we add our predicates and actions to the problem by running the following commands:

In [ ]:
problem_simple.add_fluent(At, default_initial_value=False)
problem_simple.add_fluent(On, default_initial_value=False)
problem_simple.add_fluent(Holding, default_initial_value=False)
problem_simple.add_action(move)
problem_simple.add_action(pick)
problem_simple.add_action(place)

When adding the predicates (and any other fluents) we can set their initial value already inside the `.add_fluent` method, as we did above (we don't really need to declare that the initial values are `False`, since PDDL will assume that any predicates not declared in the initial state will just be `False` there).

Next up, we need to set the initial and goal states of our problem, as we did in our PDDL problem file. In PDDL, we did so as follows:
```
	(:init (On Apple Shelf)(At Robot Table))

	(:goal (On Apple Table))
```
With the UPF, we can do this as so:

In [ ]:
problem_simple.set_initial_value(At(robot0,table), True)
problem_simple.set_initial_value(On(apple0,shelf), True)
problem_simple.add_goal(On(apple0,table))

At this point, our problem is completely defined. If we'd like to check any of the properties of our problem (i.e. we want to know what actions it contains or what the initial state is, for example), we can use the methods of the `Problem` class to do just that. For example:

In [ ]:
problem_simple.actions

In [ ]:
problem_simple.initial_values

If we want to write the problem that we've defined using the UPF to PDDL domain and problem files (this can help us ensure that we've represented our problem as we intended to), we can run the following command:

In [ ]:
w = PDDLWriter(problem_simple)
w.write_domain('SimpleExample/domain_simple_upf.pddl')
w.write_problem('SimpleExample/problem_simple_upf.pddl')

Looking inside the PDDL files that UPF produced, we can see that they appear quite similar (at least in structure) to the ones we had used back in the second tutorial. You'll notice that the `:requirements` were automatically added by the UPF without us having to explicitly declare them - this is a nice advantage of this framework. In fact, we can see our problem kind directly in Python using the `.kind` method on our `Problem` object as follows:

In [ ]:
problem_simple.kind

You should see the following *features* inside the `ProblemKind` object: `NEGATIVE_CONDITIONS` (equivalent to the `:negative-preconditions` requirement), `ACTION_BASED` (equivalent to the `:strips` requirement), and `FLAT_TYPING` (equivalent to the `:typing` requirement).

Finally, we'll want to actually solve our planning problem. To do so, we first instantiate a planner using the `OneshotPlanner` class (we can choose which planner we'd like to use, or we can let the UPF decide for us automatically), then we use its `.solve` method on our problem to produce a result (which is stored in an object of type `PlanGenerationResult`), and finally we can access the plan by calling the result's `.plan` attribute (which is of type `SequentialPlan` in this case). These steps will look as follows in Python:

In [ ]:
FD_planner = OneshotPlanner(name='fast-downward')
result_simple = FD_planner.solve(problem_simple)
plan_simple = result_simple.plan
print(plan_simple)

As expected, we obtained the same plan as we did back in Tutorial 2. The actions in this plan are contained in its `.actions` attribute, which can be called as follows:

In [ ]:
plan_simple.actions

You'll notice that we set the name for our planner to be `fast-downward` - this tells the UPF to use the [Fast Downward](https://www.aaai.org/Papers/JAIR/Vol26/JAIR-2606.pdf) planner, which we briefly mentioned in last week's tutorial. The UPF has a handful of planners already integrated for our convenience, such as [Pyperplan](https://github.com/aibasel/pyperplan), [Tamer](https://tamer.fbk.eu), and [LPG](https://lpg.unibs.it/lpg/). We can ask the UPF to automatically choose a planner for us based on our problem kind (i.e. what features we require) using the following commands:

In [ ]:
up.shortcuts.get_env().credits_stream = None
with OneshotPlanner(problem_kind=problem_simple.kind) as planner:
    result_simple = planner.solve(problem_simple)
    plan_simple = result_simple.plan
    if plan_simple is not None:
        print("%s returned:" % planner.name)
        print(plan_simple)
    else:
        print("No plan found.")

In this case the UPF automatically chose Fast Downward again, and returns the same plan we saw earlier. We can (optionally) validate the plan we've obtained, if we need further certainty that it is indeed valid (or if we obtained a plan from somewhere else and we want to check that it's valid for our problem), using the following commands:

In [ ]:
with PlanValidator(problem_kind=problem_simple.kind) as validator:
    assert validator.validate(problem_simple, plan_simple)
    print('Plan is valid!')

Finally, I've added the code for this problem as a `.py` file under the folder `Simple Example`, and you can run it directly from there using the following command:

In [ ]:
from SimpleExample import simple_upf
simple_upf.main()

<p align="center">
  <img src="Graphics/UPF2.png" width=700/>
</p>

For more information on the UPF, feel free to visit the [GitHub repository](https://github.com/aiplan4eu/unified-planning) and the [documentation site](https://unified-planning.readthedocs.io/en/latest/index.html), and you can check out the [AIPlan4EU GitHub page](https://github.com/aiplan4eu) to learn more about the planners that are currently availble to use with the UPF (you can also use other planners that are not integrated yet, but we will try to avoid doing so for now). There are some practice Jupyter notebooks [here](https://github.com/aiplan4eu/unified-planning/blob/master/notebooks/README.md) that might also be of use to you.

## (2) Ex: The Googazon Drone Delivery Service

Now let's try to tackle a more complex planning problem, in order to get a better idea of the utility of the UPF.

### (2.1) Problem Statement
In Lecture 3, you were introduced to **Googazon**, the largest (fictitious) internet company in the world, who just launched a new drone delivery service. Googazon currently has three drones and two warehouses, and each warehouse stores either milk or sugar. The delivery service currently serves 9 customers, who from time to time want either milk, sugar, or both. At a certain point in time, three customers make orders through Googazon's service, and so Googazon must plan the most efficient way (least number of total actions) to get the goods from their warehouses to their customers using the drones (who can only carry one good each at a time). We can visualize the setting of the problem as follows:
<p align="center">
  <img src="Graphics/Googazon.png" width=1100 />
</p>

### (2.2) Solution
We will discuss the solution together in class. The solution `.py` file for this problem can be found in the `Drones` folder, and it can be run here using the following commands:

In [ ]:
from Drones import drones_upf
drones_upf.main()

Notice that the planner we used here is not the regular Fast Downward planner, but rather an optimized version of it that is guaranteed to return an optimal solution (in terms of smallest number of total actions) if one exists. We can run this variant by defining our planner as follows: `planner = OneshotPlanner(name='fast-downward-opt')`.

## (3) Ex: Tower of Hanoi

In this part, you will try to model the **Tower of Hanoi** problem that we saw in Tutorial 3 using the UPF. This will serve as one of the four questions on Homework #1, so feel free to get a head start now! As a reminder, this problem was defined as follows:

### (3.1) Problem Statement
The Tower of Hanoi is a mathematical puzzle where we have 3 pegs and $N$ disks, and initially all of the disks are stacked in decreasing value of diameter on the first peg (the smallest disk is placed on the top). The objective of the puzzle is to move the entire stack of disks to the third peg using the least number of moves (which was actually shown to be $2^{N}-1$) while obeying the following rules:
* Only one disk can be moved at a time
* Each move consists of taking the upper disk from one of the stacks and placing it on top of another stack (so a disk can only be moved if it's the uppermost disk on a stack)
* No disk may be placed on top of a smaller disk

The problem for $N=2$ looks as follows:
<p align="center">
  <img src="Graphics/Hanoi.png" width=1000 />
</p>

Try to solve the $N=2$ case first, and then see if you can solve the higher level cases. The PDDL files for the $N=2$ case are available for you to look at in the `TowerOfHanoi` folder, though I recommend that you try to solve the problem without looking at them (at least initially)!

## (4) Ex: The Travelling Jedi Problem

In this part, you will try to model the **Travelling Jedi Problem** (an unweighted [Travelling Salesman Problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem)) that we saw in Tutorial 3 using the UPF. This will serve as one of the four questions on Homework #1, so feel free to get a head start now! As a reminder, this problem was defined as follows:

### (4.1) Problem Statement
Luke is taking a roadtrip with his pals aboard the Millennium Falcon, and he wants to visit a bunch of different planets, moons, and space stations across the galaxy. He will start from Tatooine, and he wants to visit every location exactly once before returning to Tatooine again. He asks the Falcon's computer (known as the Millennium Collective) to plan such a route for him, without regard to the actual geometric distance between the locations (we will deal with a weighted version of the TSP later on). The following figure shows the locations Luke would like to visit, as well as the routes that connect them to each other:
<p align="center">
  <img src="Graphics/StarWarsTJP.png" width=600 />
</p>

## (5) Conclusion

In this tutorial, we:
* Introduced the Unified Planning Framework (UPF) library, to facilitate creating PDDL problems with Python
* Got some more hands-on experience working with PDDL problem representation, as well as with Python and the UPF
* Revisited some classical planning problems like the Tower of Hanoi and the Travelling Jedi Problem using the UPF (which will also be part of the first homework)

Next week we will begin our discussion on temporal planning and scheduling, and we'll see more applications of the Unified Planning Framework.

#### ***Credit:** This tutorial was written by Yotam Granov, Winter 2022.*

### **References**

[1] AIPlan4EU. ["The Unified Planning Library"](https://unified-planning.readthedocs.io/en/latest/index.html), 2021. GitHub: https://github.com/aiplan4eu/unified-planning